In [42]:
import pandas as pd
import numpy as np

In [43]:
def payer_category(x):
    if 'Maximum Fee' in x:
        return 'max'
    elif 'Minimum Fee' in x:
        return'min'
    elif 'Cash Price' in x:
        return 'cash'
    elif 'Gross' in x:
        return 'gross'

In [90]:
folder = '.\\input_files\\'
file = '460726303_MedStar Southern Maryland Hospital Center_Standard Charges.xlsx'

df = pd.read_excel(folder + file, skiprows=3)

In [46]:
df.rename(columns={
    'Charge Description Master Detail': 'description',
    'REV/CPT/HCPCS': 'code',
    'Payor': 'payer',
    'Plan': 'plan',


}, inplace=True)

In [47]:
df.dropna(subset=['payer'], inplace=True)

In [48]:
cols = df.columns.tolist()
id_vars = cols[:4]
value_vars = ['Negotiated Fee Inpatient', 'Negotiated Fee Outpatient']
df_payers = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_temp', value_name='standard_charge')

In [49]:
df_payers['setting'] = np.where(df_payers['payer_temp'].str.contains('Inpatient'), 'inpatient', np.where(df_payers['payer_temp'].str.contains('Outpatient'), 'outpatient', 1))
df_payers.drop('payer_temp', axis=1, inplace=True)

In [50]:
df_payers['payer_category'] = 'payer'

In [51]:
# Do non-payer columns
df.drop(['Negotiated Fee Inpatient', 'Negotiated Fee Outpatient', 'payer', 'plan'], axis=1, inplace=True)

In [52]:
cols = df.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [53]:
df['payer_category'] = df['payer'].apply(payer_category)
df['setting'] = np.where(df['payer'].str.contains('Inpatient'), 'inpatient', np.where(df['payer'].str.contains('Outpatient'), 'outpatient', 1))

In [54]:
df1 = pd.concat([df, df_payers])

In [55]:
df = df1

In [69]:
df.reset_index(drop=False, inplace=True)

In [82]:
df.drop(['level_0', 'index'], axis=1, inplace=True)

In [75]:
df.loc[df['code'].str.len() == 3, 'rev_code'] = df['code']
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']
df.loc[df['code'].str.len() == 7, 'hcpcs_cpt'] = df['code'].str[:5]
df.loc[df['code'].str.len() == 7, 'modifiers'] = df['code'].str[-2:]

In [84]:
df['code'].fillna('""', inplace=True)
df['plan'].fillna('""', inplace=True)
df['rev_code'].fillna('""', inplace=True)
df['hcpcs_cpt'].fillna('""', inplace=True)
df['modifiers'].fillna('""', inplace=True)

In [92]:
ccn = {
    '460726303': '210062',
    '520491660': '210034',
    '520591607': '210056',
    '520591685': '210024',
    '520608007': '210015',
    '520619006': '210028',
    '520646893': '210018'
}

ein = file.split('_')[0]

df['hospital_id'] = ccn[ein]

In [93]:
df

,description,code,payer,standard_charge,payer_category,setting,plan,rev_code,hcpcs_cpt,modifiers,hospital_id
0,1E ADMIN STEPDOWN BED,121,Gross Charge,307.48,gross,1,"""""",121,"""""","""""",210062
1,1S ADMIN STEPDOWN BED,121,Gross Charge,307.48,gross,1,"""""",121,"""""","""""",210062
2,1W ADMIN STEPDOWN BED,121,Gross Charge,307.48,gross,1,"""""",121,"""""","""""",210062
3,2E ADMIN STEPDOWN BED,121,Gross Charge,307.48,gross,1,"""""",121,"""""","""""",210062
4,2N ADMIN STEPDOWN BED,121,Gross Charge,307.48,gross,1,"""""",121,"""""","""""",210062
...,...,...,...,...,...,...,...,...,...,...,...
688009,BC-PHP/6 GROUP THERAPY,S0201,All Payors / Plans,166.85,payer,outpatient,All Payors / Plans,"""""",S0201,"""""",210062
688010,BC-PHP/6 IND THERAPY (45),S0201,All Payors / Plans,166.85,payer,outpatient,All Payors / Plans,"""""",S0201,"""""",210062
688011,BC-PHP/6 IND THERAPY (60),S0201,All Payors / Plans,166.85,payer,outpatient,All Payors / Plans,"""""",S0201,"""""",210062
688012,BC-PHP/6 GROUP THERAPY,S0201,All Payors / Plans,166.85,payer,outpatient,All Payors / Plans,"""""",S0201,"""""",210062
